In [ ]:
# default_exp handle_game_events

In [ ]:
#hide
from nbdev.showdoc import *

 # Chapter 05 - handle_game_events 

 > : Module that explores the common elements between all GameEvents and creates some basic general purpose functions for this type of event.

In the previous modules, I focussed on the indicators I could extract and build based on the information in a replay's PlayerStatsEvents. For the most part, these indicators revolve around economic and building indicators, which point to the game's strategic (i.e. macro) dimension. Meanwhile, beyond what a player's build orders and army composition may tangentially suggest, these indicators don't seem to reveal much about the game's tactical (i.e. micro) dimension. 
What is more, the game itself and other applications like sc2replaystats offer players little information in this regard. 

One indicator gets some attention in this respect, the average actions per minute (APM). This indicator does points how fast players can play. The assumption around this marker is that if players have high APMs, this indicates that they use many of the game commands directly to control their unit's actions. Thus, I will take this indicator into account when building the player's profiles. Thankfully, sc2reader includes the APMTracker pug-in that facilitates the collection of this information. 

The problem is that this APM tells the observer little of what actions players use and when.

In this module, I define the function ...

In [ ]:
#exporti

# Load Module's dependencies

from pathlib import Path
from pprint import pprint
from typing import *

import json
import fastcore.test as ft

import sc2reader

Beyond the module import requirements, I will also use a list of each race's unique abilities. These abilities belong to specific units or buildings but are not automatically executed. Instead, players need to select the units, order the use of the ability and, in cases, chose the ability's target. 

The following code loads a dictionary that stores the list of abilities for each race into the `ab_list` variable. 

In [ ]:
#exports
data_path = (Path(Path.cwd()/'data') 
             if Path('data').exists() else Path('../data'))

with open(data_path/'ability_list.json') as af:
    ab_lists = json.load(af)

In [ ]:
# Load teh sample replays for this notebook

rps_path = Path("./test_replays")

game_path = str(rps_path/"Jagannatha LE.SC2Replay")
single_replay = sc2reader.load_replay(game_path)
single_replay

ta_test = sc2reader.load_replay(str(rps_path/'Terran_abilities.SC2Replay'))
pa_test = sc2reader.load_replay(str(rps_path/'ProtossAbilities.SC2Replay'))

tms_test = sc2reader.load_replay(str(rps_path/'TMovesSelect.SC2Replay'))
pms_test = sc2reader.load_replay(str(rps_path/'p_move_test.SC2Replay'))

## GameEvents

From the replay, I can extract its `GameEvents` (see code below). As explained in sc2reader's documentation:
> : Game events are what the Starcraft II engine uses to reconstruct games [(Kim, 2015, p. 22)](https://sc2reader.readthedocs.io/en/latest/events/game.html).

In particular, I focus on two of the `GameEvent`'s sub-classes, `CommandEvents` and `ControlGroupEvents`.


In [ ]:
test_match = pa_test
match_ge = [event for event in test_match.events 
            if isinstance(event, sc2reader.events.game.GameEvent)]
len(match_ge)

5076

## CommandEvents

`CommandEvensts` are generated and recorded every time a player orders a unit to do something during a match. These orders include building orders, some basic common commands or the use of the player's units' special abilities. Based on these classes, I exclude building and upgrade orders from the micro-game analysis because they are already reflected on the macro-game analysis. Meanwhile, I group the common commands (i.e. move, stop, patrol, hold position, follow, collect, attack) to expose play patterns common to all units. Lastly, I review the use of special abilities because it can unveil some of the player's preferences regarding the unique potential of their play race.

The following code shows how I can use the events' attributes to classify the command events. 

> Warning: The following list of comprehensions build on each other. This relationship means that for the code to run correctly, the lists must be created following the order in which they are defined.



In [ ]:

race_abilities = ab_lists[test_match.players[0].play_race]
common_abilities = ['Attack', 
                    'Stop', 
                    'HoldPosition',
                    'Patrol',
                    'RightClick']
move_command = ['RightClick']


# Firs, I can use the class to extract all command events appart from other 
# other GameEvents.
commands = [com_e for com_e in match_ge
            if isinstance(com_e, sc2reader.events.game.CommandEvent)
            and com_e.pid == 0]

# From that first list, I extract the commands linked to special abilities
# used during the game as follows.
special_comm = [com_e for com_e in match_ge 
             if isinstance(com_e, sc2reader.events.game.CommandEvent)
             and com_e.pid == 0
             and com_e.ability_name in race_abilities
             and com_e.ability_name not in common_abilities]

# I can also extract the commands related to upgrades and tech research.
upgrades = [com_e for com_e in match_ge 
             if isinstance(com_e, sc2reader.events.game.CommandEvent)
             and com_e.pid == 0
             and com_e.has_ability
             and not com_e.ability.is_build
             and com_e.ability_name not in race_abilities
             and com_e.ability_name not in common_abilities]


# I can list the common actions related to unit direction.
common_comm = [com_e for com_e in match_ge 
             if isinstance(com_e, sc2reader.events.game.CommandEvent)
             and com_e.pid == 0
             and not com_e.ability.is_build
             and com_e.ability.name in common_abilities
             and com_e.ability_name in common_abilities
             ]

# And I can list the commands that are related to building.
# In this case, I need two lists, one that is composed of the 
# abilities that are labeled as "is_build" 
build_comm1 = [com_e for com_e in match_ge
            if isinstance(com_e, sc2reader.events.game.CommandEvent)
            and com_e.pid == 0
            and com_e.has_ability
            and not com_e.ability_name in race_abilities
            and com_e.ability.is_build]

# and one that has no linked ability, but are that construct 
# each race's vespene gas extraction facilities.
build_comm2 = [com_e for com_e in match_ge
            if isinstance(com_e, sc2reader.events.game.CommandEvent)
            and com_e.pid == 0
            and not com_e.has_ability
            and (not com_e.ability.is_build
               and 'Build' in com_e.ability_name)]



I can verify the validity of this classification by adding all the lists lengths and confirming they have the same number of elements as the `CommandEvent` list. 

In [ ]:
extras = [com_e for com_e in match_ge
        if isinstance(com_e, sc2reader.events.game.CommandEvent)
        and com_e.pid == 0
        and (com_e not in special_comm
            and com_e not in upgrades
            and com_e not in common_comm
            and com_e not in build_comm1
            and com_e not in build_comm2)]

print(f'Special abilities commands: {len(special_comm)}')
print(f'Special abilities upgrades commands: {len(upgrades)}')
print(f'Common commands: {len(common_comm)}')
print(f'Build Commands: {len(build_comm1)}')
print(f'Build Vespene Extractor Facility Commands: {len(build_comm2)}')
print(f'Extras: {len(extras)}')

command_lists=[special_comm, 
              upgrades,
              common_comm,
              build_comm1,
              build_comm2,
              extras]

sum_lists = sum([len(c_list) for c_list in command_lists])

print(f'Total sum: {sum_lists}')
print(f'Total Commands: {len(commands)}')

Special abilities commands: 64
Special abilities upgrades commands: 30
Common commands: 180
Build Commands: 78
Build Vespene Extractor Facility Commands: 3
Extras: 0
Total sum: 355
Total Commands: 355


Additionally, I run the following test to ensure that the sum above counts each element once and not double-counting some while ignoring others.

In [ ]:
lists=[special_comm, upgrades, common_comm, build_comm1, build_comm2]
repeats = []
for ind1, l1 in enumerate(lists):
    for ind2, l2 in enumerate(lists):
        if l1 != l2:
            for e in l1:
                if e in l2:
                    repeats.append(e)

ft.test_eq(len(repeats), 0)

### Classifying common commands

Beyond the command classification above, I can also sub-divide the common commands into distinct types. Upon some examination, I realise that these events relate to direct attacks, unit movement orders, or collection orders that tell units to gather Minerals or Vespene Gas. 

The following code illustrates how I can list the common commands into these categories. 


In [ ]:
attacks = [att for att in common_comm
        if att.ability.name == 'Attack']

resources = ['Mineral', 'Vespene', 'Extractor', 'Refinary', 'Assimilator']
collects = [coll for coll in common_comm
           if hasattr(coll, 'target')
           and (lambda event: any(map(lambda rsc: rsc in event.target.name, 
                                      resources))
                if hasattr(event, 'target') else True)(coll)]

follows = [foll for foll in common_comm
           if hasattr(foll, 'target')
           and not (lambda event: any(map(lambda rsc: rsc in event.target.name,
                                         resources))
                if hasattr(event, 'target') else True)(foll)]

moves_names = ['Stop', 'Patrol', 'HoldPosition', 'RightClick']
moves = [move for move in common_comm
        if move.ability_name in moves_names
        and move not in collects
        and move not in follows]


In [ ]:
extras = [ext for ext in common_comm
        if not(lambda x: any(map(lambda e_list: x not in e_list, 
                                 [attacks, moves, collects])))(ext)]

print(f'Attacks: {len(attacks)}')
print(f'Collects: {len(collects)}')
print(f'Follows: {len(collects)}')
print(f'Moves: {len(moves)}')
print(f'Extras: {len(extras)}')

command_lists=[attacks, 
              collects,
              moves,
              extras]

sum_lists = sum([len(c_list) for c_list in command_lists])

print(f'Common Commands: {len(common_comm)}')
print(f'Total sum: {sum_lists}')

ft.test_eq(len(extras),0)

Attacks: 12
Collects: 23
Follows: 23
Moves: 116
Extras: 0
Common Commands: 180
Total sum: 151


In [ ]:
lists=[attacks, collects, follows, moves, extras]

repeats = []
for ind1, l1 in enumerate(lists):
    for ind2, l2 in enumerate(lists):
        if l1 != l2:
            for e in l1:
                if e in l2:
                    repeats.append(e)

ft.test_eq(len(repeats), 0)

## CommandEvents Exportable Functions

In this section, I define this module's exportable functions. These functions use the `CommandEvent` classification discussed above to calculate several micro-game performance indicators.

First, I use the special abilities list to calculate the ratio between them and the total commands executed by the player. With this ratio, I can quantify the use of these abilities. The measurement helps me separate players that use their race's unique capabilities from those who do not.

 > Note: Following the logic set in previous modules, I define these functions to calculate all indicators for the early, mid, late and whole game. Although I do this to remain consistent, I also think this level of detail makes sense given how players have access to different units and abilities at various game stages.


I also calculate the player's first and second preferred abilities, if they use any. This qualitative piece of information can indicate some aspects of the player's tactical preferences.


Meanwhile, I define the following function to estimate a player's aggressiveness. For this purpose, the function calculates the ratio between attack commands and the number of common commands as a potential indicator of a player's aggressiveness.